# Unemployment/Labor Force Participation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

## Import Democracy Data

In [4]:
dem = pd.read_hdf("data/democracy.h5", "democracy")

## Read Unemployment

In [19]:
unemployment = pd.read_csv("data/unemployment.csv", skiprows=[0, 1, 2, 3], index_col=0).drop(["Country Code", "Indicator Name", "Indicator Code", "Unnamed: 65"], axis=1)
unemployment = unemployment.unstack().to_frame("unemployment").reorder_levels([1, 0])
unemployment.index.set_names(["country_name", "year"], inplace=True)
unemployment.sort_index(inplace=True)
unemployment

unemployment
country_name year              
Afghanistan  1960           NaN
             1961           NaN
             1962           NaN
             1963           NaN
             1964           NaN
...                         ...
Zimbabwe     2016          5.24
             2017          5.15
             2018          5.07
             2019          5.02
             2020          5.73

[16226 rows x 1 columns]

## LinReg

In [ ]:
dem_ump = dem.join()